<a href="https://www.kaggle.com/code/bhavkaur/machine-failure-baseline-eda?scriptVersionId=133889273" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

![](https://raw.githubusercontent.com/Bhavleenk/Kaggle-Notebooks/main/PS_3.17_Machine_Failure/banner.jpg)
# **Binary Classification of Machine Failures** 

# Importing Libraries

In [82]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt
import seaborn as sns

/kaggle/input/playground-series-s3e17/sample_submission.csv
/kaggle/input/playground-series-s3e17/train.csv
/kaggle/input/playground-series-s3e17/test.csv


# Reading Data

In [83]:
test_data = pd.read_csv("/kaggle/input/playground-series-s3e17/test.csv", index_col="id")
train_data = pd.read_csv("/kaggle/input/playground-series-s3e17/train.csv", index_col="id")

# Exploratory Data Analysis

In [84]:
train_data.head()

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
id,,,,,,,,,,,,,
0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [85]:
train_data.columns

Index(['Product ID', 'Type', 'Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]',
       'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'],
      dtype='object')

In [86]:
train_data.isna().sum()

Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

**There are no Null values in the dataset, so we don't need to handle Null values.**

In [87]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136429 entries, 0 to 136428
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Product ID               136429 non-null  object 
 1   Type                     136429 non-null  object 
 2   Air temperature [K]      136429 non-null  float64
 3   Process temperature [K]  136429 non-null  float64
 4   Rotational speed [rpm]   136429 non-null  int64  
 5   Torque [Nm]              136429 non-null  float64
 6   Tool wear [min]          136429 non-null  int64  
 7   Machine failure          136429 non-null  int64  
 8   TWF                      136429 non-null  int64  
 9   HDF                      136429 non-null  int64  
 10  PWF                      136429 non-null  int64  
 11  OSF                      136429 non-null  int64  
 12  RNF                      136429 non-null  int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 14.6+ MB


In [88]:
len(train_data["Product ID"].unique())
train_data.drop(['Product ID'], inplace=True, axis=1)
test_data.drop(['Product ID'], inplace=True, axis=1)

In [89]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
a=train_data['Type']
train_data['Type']=encoder.fit_transform(a)

a=test_data['Type']
test_data['Type']=encoder.transform(a)

len(train_data['Type'].unique())
train_data.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
id,,,,,,,,,,,,
0,1,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,2,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,1,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,1,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,2,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [90]:
train_data.columns = ['Type', 'Air_temp', 'Process_temp',
       'Rotational_speed', 'Torque', 'Tool_wear',
       'Machine_failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']
train_data.columns
test_data.columns=['Type', 'Air_temp', 'Process_temp',
       'Rotational_speed', 'Torque', 'Tool_wear', 'TWF',
       'HDF', 'PWF', 'OSF', 'RNF']
test_data.columns

Index(['Type', 'Air_temp', 'Process_temp', 'Rotational_speed', 'Torque',
       'Tool_wear', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'],
      dtype='object')

# Training Models

1. RandomForestClassifier
2. GradientBoostingClassifier
3. XGBClassifier

In [125]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

seed = np.random.seed(6)

In [126]:
target_column='Machine_failure'
features = [i for i in train_data.columns]
features.remove("Machine_failure")

# Entire Train DataX
X=train_data[features]
y=train_data[target_column]

In [127]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=seed, test_size=0.3)

In [128]:
rfmodel = RandomForestClassifier(random_state=seed)
rfmodel.fit(X_train, Y_train)

RandomForestClassifier()

In [129]:
gbmodel = GradientBoostingClassifier(random_state=seed)
gbmodel.fit(X_train, Y_train)

GradientBoostingClassifier()

In [130]:
xgmodel = XGBClassifier(random_state=seed)
xgmodel.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

# Calculating Accuracy Score

In [131]:
rfpred = rfmodel.predict(X_test)
print("ROC Area Under Curve of RandomForestClassifier : ",roc_auc_score(Y_test, rfpred))

ROC Area Under Curve of RandomForestClassifier :  0.8851223753212655


In [132]:
gbpred = gbmodel.predict(X_test)
print("ROC Area Under Curve of GradientBoostingClassifier : ", roc_auc_score(Y_test, gbpred))

ROC Area Under Curve of GradientBoostingClassifier :  0.8851471990726308


In [133]:
xgpred = xgmodel.predict(X_test)
print("ROC Area Under Curve of XGBClassifier : ", roc_auc_score(Y_test, xgpred))

ROC Area Under Curve of XGBClassifier :  0.8866727629181647


Baseline XGB Model gives the best accuracy

# Training on Full Train Dataset

In [134]:
xgmodel.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

# Creating 'submission.csv' File

In [135]:
preds = xgmodel.predict(test_data)

In [136]:
submit = test_data.copy()
submit['Machine failure'] = preds
# submit['id'] = test_data.index 

In [137]:
submit.columns

Index(['Type', 'Air_temp', 'Process_temp', 'Rotational_speed', 'Torque',
       'Tool_wear', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF', 'Machine failure'],
      dtype='object')

In [138]:
col=[i for i in submit.columns]
# col.remove('id')
col.remove('Machine failure')
submit.drop(col,inplace=True, axis=1)

In [139]:
submit.head()

,Machine failure
id,
136429,0
136430,0
136431,0
136432,0
136433,0


In [140]:
submit.to_csv("submission.csv", index=True, header=True)